Acceleromter Stuff
====
Initial exploration

### Prerequisites
The right version of python and various packages and probably a GPU

### where the data is
I've copied a single accelerometer file into the `data/` directory to do this initial exploration.

This isn't a viable long-term solution, but downloading from RDSF takes forever

I've also mounted the RDSF files on my `Z:` drive - change this in `userconf.yaml` if you've done something different

One can also mount the files in the right place with
```
sudo mount -t cifs -o username=mh19137,workgroup=UOB //rdsfcifs.acrc.bris.ac.uk/SEACOCH20 /mnt/z/
```

In [ ]:
# I've written some modules
from ema import read, parse, analysis

### Choose a participant and check if they consented

In [ ]:
device_id = "6012582"
recording_id = "0000291121"
participant_id = "20029"

if not read.consented(participant_id):
    raise ValueError(f"Participant {participant_id} didn't consent")

In [ ]:
# Find the accelerometer file
import pathlib

filename = f"{device_id}_{recording_id}-{participant_id}.cwa"
filepath = pathlib.Path(rf"data/{filename}")

# TODO add some code to look for the right files in all the "Week X" folders

assert filepath.exists()

In [ ]:
""" Find the smartwatch data from the combined smartwatch file """
meal_df = read.meal_info(participant_id)
allowed_meal_types = {"Snack", "Drink", "Meal", "No food/drink"}
meal_df = parse.extract_meals(meal_df, allowed_meal_types, verbose=True)

In [ ]:
""" Read the accelerometry file """
samples = read.accel_info(str(filepath))

In [ ]:
# Use time as the index
samples.set_index("time", inplace=True, verify_integrity=False)

In [ ]:
""" Find the total magnitude of acceleration """
import numpy as np

samples["magnitude"] = np.sqrt(
    samples["accel_x"] ** 2 + samples["accel_y"] ** 2 + (samples["accel_z"] - 1) ** 2
)

In [ ]:
import matplotlib.pyplot as plt

fig, axis = plt.subplots(figsize=(18, 3))

# Plot Accelerometry
axis.plot(samples.index, samples["magnitude"], linewidth=0.3, color="k")

In [ ]:
from pandas import Timedelta, to_datetime

# Find meal times
meal_times = meal_df["date"].map(str) + meal_df["timestamp"]

# Find an hour slot before each meal
ends = to_datetime(meal_times, format=r"%d%b%Y%H:%M:%S")
starts = ends - Timedelta(1, "hour")

In [ ]:
# Plot the total acceleration in an hour slot before each was recorded
import pathlib
from tqdm import tqdm

from ema import util

# Make interactive plotting work
import os
import PyQt6.QtCore
os.environ["QT_API"] = "pyqt5"

%matplotlib qt


keep = meal_df["meal_type"] == "Meal"
n_meals = np.sum(keep)

plot_dir = pathlib.Path("plots/")

if not plot_dir.is_dir():
    plot_dir.mkdir()

# To explore some of the time series...
for i, (start, end) in tqdm(enumerate(zip(starts[keep], ends[keep]))):
    # First few are empty
    if i < 5:
        continue

    accel = samples.loc[start:end]["accel_x"]

    # 100Hz => dx=0.01
    vely = util.integrate(accel, dx=0.01)
    posn = util.integrate(vely, dx=0.01)

    fig, axes = plt.subplots(3, 1, figsize=(12, 6))

    for axis, label, data in zip(axes, ("Posn", "Vely", "Accn"), (posn, vely, accel)):
        axis.plot(samples[start:end].index, data)
        axis.set_ylabel(label)

    break


In [ ]:
# To explore some of the time series...
from tqdm import tqdm
%matplotlib qt

def smooth(x, w):
    return np.convolve(x, np.ones(w), "same") / w

keep = meal_df["meal_type"] == "Meal"

for i, (start, end) in tqdm(enumerate(zip(starts[keep], ends[keep]))):
    # First few are empty
    if i < 5:
        continue

    slice = samples.loc[start:end]
    accel = slice["accel_z"]

    smoothed = smooth(accel, 1)

    smoothed = smoothed - np.mean(smoothed)

    # 100Hz => dx=0.01
    vely = util.integrate(smoothed, dx=0.01)
    vely -= np.mean(vely)

    posn = util.integrate(vely, dx=0.01)
    posn -= np.mean(posn)

    fig, axes = plt.subplots(3, 1, figsize=(12, 6))

    axes[2].plot(slice.index, accel, linewidth=0.5, color="k")
    axes[2].plot(slice.index, smoothed, linewidth=1.5, linestyle="--", color="r")

    axes[1].plot(slice.index, vely)
    axes[0].plot(slice.index, posn)

    for axis, label, data in zip(axes, ("Posn", "Vely", "Accn"), (posn, vely, accel)):
        axis.plot(samples[start:end].index, data)
        axis.set_ylabel(label)
        axis.axhline(0)

    break